In [1]:
from numpy import array
import string
from pickle import dump
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Lambda
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from pickle import load
import re

def load_doc(filename):
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

Using TensorFlow backend.


In [2]:
#preprocess each word
def parse_word(word):
    word = word.lower()
    #delete puncuations
    if word.endswith(','): 
        word = word[:-1]
    if word.endswith('.'):
        word = word[:-1]
    if word.endswith('?'): 
        word = word[:-1]
    if word.endswith('!'): 
        word = word[:-1]
    if word.endswith('?'): 
        word = word[:-1]
    if word.endswith(';'): 
        word = word[:-1]
    if word.endswith(':'): 
        word = word[:-1]
    if word.endswith(')'): 
        word = word[:-1]
    if word.endswith("'"):
        if word != "th'":
            word = word[:-1]
    if word.startswith('('):
        word = word[1:]
    if word.startswith("'"):
        word = word[1:]
    return word

def get_text(filename):    
    with open(filename) as f:
        text = ''
        for line in f:
            if line.split() == []:
                continue
            if line.split()[0].isdigit():
                continue   
            #add the words in the poem
            for word in line.split():
                #preprocess
                #word = parse_word(word) #somehow we want to predict puncutation here
                text += word + ' '
    f.close()
    return text

fp = r'C:\\Users\\happy\\Desktop\\project 3\\Official_Release\\data\\shakespeare.txt'
raw_text = get_text(fp)
#print(raw_text)

#not sure if i should train using all poems or one poem by one

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus() #sadly no gpu

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14474103071045859987
]






[]

In [4]:
#organize into sequence of characters
length = 40
sequence = list()
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i+1]
    sequence.append(seq)
print("total sequences: %d" % len(sequence))

total sequences: 93634


In [5]:
#save stuff to file for some reason, otherwise might have memory issue
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close

#save shit to file
out_f = 'dark_char.txt'
save_doc(sequence, out_f)

In [6]:
#load data back
in_f = 'dark_char.txt'
raw = load_doc(in_f)
lines = raw.split('\n')
print(lines[0])

#encode sequence
chars = sorted(list(set(raw)))
print(chars)

mapping = dict((c,i) for i, c in enumerate(chars))
print(mapping)

encoded_sequence = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    encoded_sequence.append(encoded_seq)
print(encoded_sequence[0])

From fairest creatures we desire increase
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'A': 12, 'B': 13, 'C': 14, 'D': 15, 'E': 16, 'F': 17, 'G': 18, 'H': 19, 'I': 20, 'J': 21, 'K': 22, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'Y': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'j': 44, 'k': 45, 'l': 46, 'm': 47, 'n': 48, 'o': 49, 'p': 50, 'q': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'v': 56, 'w': 57, 'x': 58, 'y': 59, 'z': 60}
[17, 52, 49, 47, 1, 40, 35, 43, 52, 39, 53, 54, 1, 37, 52, 39, 35, 54, 55, 52, 39, 53, 1, 57, 39, 1, 38, 39, 5

In [7]:
#vocab size
vocab_size = len(mapping)
print('vocab size is : {}'.format(vocab_size))

vocab size is : 61


In [8]:
encoded_sequence = array(encoded_sequence)
X, y = encoded_sequence[:,:-1], encoded_sequence[:,-1]

#dont understand here....looks like made it into tensor
encoded_sequence = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(encoded_sequence)
y = to_categorical(y, num_classes=vocab_size)

print(X.shape)

(93634, 40, 61)


In [9]:
import os
import tensorflow as tf
#removing those fucking annoying deprecated warning messages from tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def make_model(X, temperature=1):
    model = Sequential()
    #layer of 100-200 LSTM suggested by spec
    model.add(LSTM(100, input_shape=(X.shape[1],X.shape[2])))
    model.add(Lambda(lambda x: x * temperature)) #temperature layer as suggested in spec
    #model.add(Dense(64, activation='relu')) #can add this layer for additional work if have time
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    #plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

def gen_poem(model, mapping, seq_length, start_text, n_chars):
    in_text = start_text
    #modifying some shit since we want to print exactly 14 lines
    for _ in range(n_chars): 
        #dont want to do while loop, if we get super long sentence, we are basically fucked
        encoded = [mapping[char] for char in in_text]
        
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        
        encoded = to_categorical(encoded, num_classes=len(mapping))
        
        #predict the next char
        y_pred = model.predict_classes(encoded)
        
        #get the char for y_pred
        char_pred = ''
        for char, index in mapping.items():
            if index == y_pred:
                char_pred = char
                break
        
        #append to our in_text
        in_text += char_pred
        
        #do a '\n' check
        count = len(re.findall("\n", in_text))
        if count == 14:
            return in_text
    print('no 14 lines...failed')
    return in_text
    

In [32]:
###########temprature = 1.5
rnn = make_model(X, temperature=1.5)
#keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
rnn.fit(X, y, epochs=10, batch_size=256, callbacks=[EarlyStopping(monitor='acc', patience=2, verbose=0, mode='auto')])
rnn.save('rnn.h5')
dump(mapping, open('mapping.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               64800     
_________________________________________________________________
lambda_4 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 61)                6161      
Total params: 70,961
Trainable params: 70,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
93634/93634 [==============================] - 42s 453us/step - loss: 2.8186 - acc: 0.2435
Epoch 2/10
93634/93634 [==============================] - 36s 383us/step - loss: 2.2773 - acc: 0.3504
Epoch 3/10
93634/93634 [==============================] - 39s 414us/step - loss: 2.1427 - acc: 0.3831
Epoch 4/10
93634/93634 [==============================] - 33s 347us/step - loss: 2.0625 - acc: 0

In [33]:
rnn150 = load_model('rnn.h5')
mapping = load(open('mapping.pkl', 'rb'))
fugma = "shall i compare thee to a summer's day?\n"
print(gen_poem(rnn150, mapping, 40, fugma, 1000))

no 14 lines...failed
shall i compare thee to a summer's day?
The so the self the with the self the will deart, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self the with the self the with thee, And the self th

In [11]:
###########temprature = 0.75
rnn_075 = make_model(X, temperature=0.75)
#keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
rnn_075.fit(X, y, epochs=20, batch_size=256, callbacks=[EarlyStopping(monitor='acc', patience=2, verbose=0, mode='auto')])
rnn_075.save('rnn_075.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               64800     
_________________________________________________________________
lambda_2 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 61)                6161      
Total params: 70,961
Trainable params: 70,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
93634/93634 [==============================] - 98s 1ms/step - loss: 2.9952 - acc: 0.1917
Epoch 2/20
93634/93634 [==============================] - 35s 369us/step - loss: 2.6317 - acc: 0.2971
Epoch 3/20
93634/93634 [==============================] - 37s 398us/step - loss: 2.3890 - acc: 0.33130s - loss: 2.3893 - acc: 0.331
Epoch 4/20
93634/93634 [==============================] - 34s 363us/

In [12]:
rnn_075 = load_model('rnn_075.h5')
mapping = load(open('mapping.pkl', 'rb'))
fugma = "shall i compare thee to a summer's day?\n"
print(gen_poem(rnn_075, mapping, 40, fugma, 1000))

no 14 lines...failed
shall i compare thee to a summer's day?
 the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the stor

In [39]:
###########temprature = 0.25
rnn_025 = make_model(X, temperature=0.25)
#keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
rnn_025.fit(X, y, epochs=20, batch_size=128, callbacks=[EarlyStopping(monitor='acc', patience=2, verbose=0, mode='auto')])
rnn_025.save('rnn_025.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100)               64800     
_________________________________________________________________
lambda_8 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 61)                6161      
Total params: 70,961
Trainable params: 70,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
93634/93634 [==============================] - 86s 917us/step - loss: 3.0697 - acc: 0.1869
Epoch 2/20
93634/93634 [==============================] - 79s 849us/step - loss: 3.0139 - acc: 0.1877
Epoch 3/20
93634/93634 [==============================] - 82s 877us/step - loss: 2.9673 - acc: 0.1877
Epoch 4/20
93634/93634 [==============================] - 89s 953us/step - loss: 2.9022 - acc: 0

In [40]:
rnn_025 = load_model('rnn_025.h5')
mapping = load(open('mapping.pkl', 'rb'))
fugma = "shall i compare thee to a summer's day?\n"
print(gen_poem(rnn_025, mapping, 40, fugma, 1000))

no 14 lines...failed
shall i compare thee to a summer's day?
 the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th